
Привет очередной раз, меня зовут Люман Аблаев. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на красные комментарии ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:


<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление</font>
* Приятно было снова проверять твою работу
- Я постарался оставить полезные советы, надеюсь они тебе пригодятся.
- Увы, я обнаружил недочеты с которыми нужно будет поработать и их устранить.
- Надеюсь у тебя получится, уделить работе больше внимания и привести ее в порядок
- Жду обновленную работу!



# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import time

<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты  на месте
</div>

## Подготовка данных

In [ ]:
# Загрузка данных
data = pd.read_csv('/datasets/autos.csv')


<div class="alert alert-block alert-danger">
<b>Ошибка:</b> 
    
1) Прежде чем переходить к обработке данных, их нужно изучить... посмотреть пропуски, статистики, дубликаты, описать свои наблюдения и только затем переходить к их обработке.
    
2) Все комментарии сопровождающие твои действия должны находится в отдельных markdown ячейках. Комментарии в коде предназначены для разработчиков и не очень там уместны.
    

In [ ]:
## Удаление неинформативных признаков
data = data.drop(columns=['DateCrawled', 'LastSeen', 'DateCreated', 'NumberOfPictures', 'PostalCode'])


<div class="alert alert-block alert-success">
<b>Успех:</b> Согласен с перечнем неинформативных
</div>

In [ ]:
# Заполнение пропущенных значений
data = data.dropna()

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Не согласен с таким способом обработки пропусков, теряется огромное количество данных
</div>


In [ ]:
# Обработка аномалий в столбце 'Power'
data = data[(data['Power'] >= 10) & (data['Power'] <= 500)]

<div class="alert alert-block alert-info">
<b>Совет:</b>  Вопрос на подумать: Как думаешь а машины с 0-ой мощностью могут быть просто без двигателя?
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Также есть аномалии в целевом признаке и годе регистрации, которые нужно проанализировать и если нужно обработать
</div>


In [ ]:
# Подготовка признаков и целевой переменной
X = data.drop(columns=['Price'])
y = data['Price']

In [ ]:
# Преобразование категориальных признаков в числовые
le = LabelEncoder()
X['VehicleType'] = le.fit_transform(X['VehicleType'])
X['Gearbox'] = le.fit_transform(X['Gearbox'])
X['Model'] = le.fit_transform(X['Model'])
X['FuelType'] = le.fit_transform(X['FuelType'])
X['Brand'] = le.fit_transform(X['Brand'])
X['Repaired'] = le.fit_transform(X['Repaired'])



<div class="alert alert-block alert-danger">
    

<b>Ошибка:</b>  
    
1) LabelEncoder - инструмент предназначенный для кодирования целевого признака (если он категориальный), поэтому для обучающих признаков его лучше не использовать.
  
2) Отмечу, что encoder правильно применять после разбиения данных и обучать только на обучающей выборке, для остальных выборок просто использовать transform. Примеры использования с объяснениями можно найти посмотреь https://colab.research.google.com/drive/1_gAMXcQKoCShB_l8FNtYEejMnosm9mvt?usp=sharing 

И не забывай использовать параметр `handle_unknown`
</div>


In [ ]:
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Обучение моделей

In [ ]:
# Обучение модели LightGBM
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'min_child_samples': 20,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'n_jobs': -1,
    'random_state': 42
}

start_time = time.time()
lgb_train = lgb.Dataset(X_train, y_train)
lgb_model = lgb.train(params, lgb_train, num_boost_round=100)
end_time = time.time()
training_time = end_time - start_time


In [ ]:
# Время предсказания модели
start_time = time.time()
y_pred = lgb_model.predict(X_test)
end_time = time.time()
prediction_time = end_time - start_time



<div class="alert alert-block alert-danger">
   
<b>Ошибка:</b>
   
- Тестовую выборку мы не должны использовать при вычислении метрик для анализа. Она должна использоваться только для единственной лучшей модели после анализа  в шаге с тестированием.
- У нас заданы 3 метрики интересующие заказчика: `время обучения: .fit()`, `скорость предсказания: .predict()` и `качество: RMSE`. Эти метрики мы получаем и анализируем отдельно друг от друга (не на тестовой выборке и без учета времени подбора  гиперпаметров) 
- Получить их можем либо с помощью валидационной выборки либо используя кросс-валидационные методы (в GridSearchCV например есть все данные метрики в аттрибуте cv_results_).   В случае если не используется ни gridsearch ни валидационная выборка, то можно время предсказания замерить на тренировочной (качество так замерить нельзя).
- Потом проводится анализ и выбирается ОДНА наилучшая модель.
- И только затем для ОДНОЙ наилучшей модели  проводится тестирование.
  
Исправь пожалуйста это здесь и везде внизу
</div>


## Анализ моделей



<div class="alert alert-block alert-danger">
   
<b>Ошибка:</b> 
    
1) Нужно рассмотреть еще хотя бы 1 модель не из семейства бустингов (желательно не линейную регрессию)
    
2) Для моделей нужно посчитать три метрики: `время обучения: .fit()`, `скорость предсказания: .predict()` и `качество: RMSE`. Их нужно явно проанализировать и выбрать лучшую модель. И только затем переходить к тестированию лучшей модели
</div>


In [ ]:
# Качество модели
rmse = mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
print("Training Time:", training_time)
print("Prediction Time:", prediction_time)
print("RMSE:", rmse)



Исходя из проведенного анализа, получены следующие результаты:

- Время обучения модели составило 75.89 секунды, что является приемлемым временем для обучения модели на данном объеме данных.

- Время предсказания модели составило 0.40 секунды, что также является достаточно быстрым для предсказания цены автомобиля.

- Качество предсказания модели оценивается с помощью метрики RMSE, которая показывает среднеквадратичное отклонение предсказанной цены от фактической. В данном случае, RMSE равно 1726.88, что говорит о том, что модель может предсказывать рыночную стоимость автомобиля с некоторой погрешностью около 1726.88 единиц.

Исходя из этих результатов, можно сделать вывод, что модель достаточно быстрая в обучении и предсказании, и имеет приемлемое качество предсказания. Она может быть использована в приложении для определения рыночной стоимости автомобилей и привлечения новых клиентов.




<div class="alert alert-block alert-info">
<b>Совет:</b> 

Также если говорить, что можно ещё улучшить в подобных проектах, то я бы выделил такие моменты:<br>
    
1) Напомню, что для понимания, а какие в итоге факторы важны при моделировании, можно выводить их важность, использую feature_importances_, ну и график заодно. 
    
2) У нас разный возраст машин. Есть гипотеза, что для разных возрастов - своё ценообразование. Поэтому, можно попробовать ввести фактор "тип возраста" (ретро, супер-ретро, старая, новая... надо подумать..)..<br>

3) У некоторых моделей семейства бустингов, есть внутренний метод кодировки данных, который хорошо было бы попробовать
</div>

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнена загрузка и подготовка данных
- [ ]  Выполнено обучение моделей
- [ ]  Есть анализ скорости работы и качества моделей